# Detectarea fețelor în imagini folosind o rețea neuronală cu o matrice de detecție

Acest notebook demonstrează procesul de inferență utilizând modele deja antrenate. Rețelele utilizează un bloc convoluțional pentru extragerea caracteristicilor imaginilor și un component complet conectat pentru a produce o matrice de n x n blocuri, fiecare dintre acestea având o probabilitate între 0 și 1 de a conține un fragment de față.

Modelele sunt mult prea mari pentru a fi încărcate interactiv, folosim posibilitatea de a monta google drive în mașina virtuală care execută codul

In [ ]:
import google.colab
google.colab.drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


## Importul bibliotecilor folosite

In [ ]:
import sys
import os
import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from google.colab import files
from IPython.display import display
import gc

## Verificarea corectitudinii căilor către fișierele care conțin modele antrenate

In [ ]:
model_dir = '/content/gdrive/MyDrive/Facultate Informatica/Profesor/2024 - 2025/Biometrie/Sisteme expert si metode biometrice in securitatea informatiei/ColabMount/Models/Face/GridNetwork/224/GridSimpleCNNModel'
yolo_model_dir = '/content/gdrive/MyDrive/Facultate Informatica/Profesor/2024 - 2025/Biometrie/Sisteme expert si metode biometrice in securitatea informatiei/ColabMount/Models/Face/GridNetwork/224/GridYOLOCNNModel'

In [ ]:
if os.path.isdir(model_dir):
  print("Model directory exists")
else:
  print("Model directory does not exist")

if os.path.isdir(yolo_model_dir):
  print("Model directory exists")
else:
  print("Model directory does not exist")

Model directory exists
Model directory exists


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Funcții utile

### Functia de transformare

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

### Functia care deseneaza un grid cu probabilitatile obtinute

In [ ]:
def getGridImage(original_image, output, grid_size=7):
    resized_confidence_map = output.cpu().detach().numpy()
    resized_confidence_map = resized_confidence_map.astype(np.float32)
    resized_confidence_map = np.transpose(resized_confidence_map)
    grid_map = resized_confidence_map


    image_width, image_height = original_image.size
    rows, cols = grid_size, grid_size
    cell_width = image_width / cols
    cell_height = image_height / rows

    # for i in range(rows):
    #     for j in range(cols):
    #         print(f"{grid_map[i, j]:.1f} ", end="")
    #     print("")

    # Create the figure
    image_tmp = original_image.copy()
    fig, ax = plt.subplots(figsize=(image_width / 100, image_height / 100), dpi=100)
    ax.imshow(image_tmp)
    masked_grid = np.ma.masked_where(grid_map == 0, grid_map)
    ax.imshow(masked_grid, cmap='viridis', alpha=0.5, interpolation='nearest', vmin=0, vmax=1, extent=[0, image_width, image_height, 0])

    # Grid lines
    for i in range(rows + 1):
        ax.axhline(i * cell_height, color='black', linewidth=1, alpha=0.5)
    for j in range(cols + 1):
        ax.axvline(j * cell_width, color='black', linewidth=1, alpha=0.5)

    # Grid cell values
    for i in range(rows):
        for j in range(cols):
            if grid_map[i, j] > 0.1:
                ax.text((j + 0.5) * cell_width, (i + 0.5) * cell_height, f'{grid_map[i, j]:.1f}', ha='center', va='center', color='white', fontsize=10)

    ax.axis('off')  # Hide axes

    # Convert figure to PIL image
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    buf.seek(0)
    pil_img = Image.open(buf).convert('RGB')
    plt.close(fig)

    return pil_img

## Definirea modelelor rețelelor neuronale

### Modelul convoluțional simplu

In [ ]:
class GridSimpleCNNModel(nn.Module):
    def __init__(self, griddim=7, num_classes=1, image_size = 224):
        super(GridSimpleCNNModel, self).__init__()
        self.griddim = griddim
        self.num_classes = num_classes
        self.image_size = image_size

        # Define the convolutional layers in a Sequential block
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3), nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (64, 56, 56)
            nn.Conv2d(64, 192, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (192, 28, 28)
            nn.Conv2d(192, 256, kernel_size=3, padding=1), nn.LeakyReLU(0.1),  # Output: (256, 28, 28)
            nn.Conv2d(256, 512, kernel_size=3, padding=1), nn.LeakyReLU(0.1),  # Output: (512, 28, 28)
            nn.MaxPool2d(kernel_size=2, stride=2)  # Output: (512, 14, 14)
        )

        # Define the fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(100352, 4096),  # Fully connected layer after conv layers
            nn.ReLU(),
            nn.Linear(4096, self.griddim ** 2),  # Output grid cells
            nn.Sigmoid()  # Sigmoid to output values between 0 and 1
        )

    def forward(self, x):
        # Apply the convolutional layers
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        x = x.view(x.size(0), self.griddim, self.griddim)
        return x


### Modelul convoluțional cu un bloc de tip YOLO

Spre deosebire de modelul simplu, acesta conține un bloc convoluțional mult mai complex, inspirat din YOLO (You Only Look Once) - https://arxiv.org/abs/1506.02640

In [ ]:
class GridYOLOCNNModel(nn.Module):
    def __init__(self, griddim=7, num_classes=1, image_size = 224):
        super(GridYOLOCNNModel, self).__init__()
        self.griddim = griddim
        self.num_classes = num_classes
        self.image_size = image_size

        # Define the convolutional layers in a Sequential block
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3), nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (64, 112, 112)
            nn.Conv2d(64, 192, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (192, 56, 56)
            nn.Conv2d(192, 256, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (512, 28, 28)
            nn.Conv2d(512, 1024, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (1024, 14, 14)
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1), nn.LeakyReLU(0.1),  # Output: (1024, 7, 7)
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
        )


        # Define the fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(50176, 4096),  # Fully connected layer after conv layers
            nn.ReLU(),
            nn.Linear(4096, self.griddim ** 2),  # Output grid cells
            nn.Sigmoid()  # Sigmoid to output values between 0 and 1
        )

    def forward(self, x):
        # Apply the convolutional layers
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        x = x.view(x.size(0), self.griddim, self.griddim)
        return x


## Incărcarea imaginii de test

In [ ]:
print("Incarcati o imagine de test")
uploaded_test_image = files.upload()  # Allows user to upload a file
# Get the first uploaded file
uploaded_test_image_path = list(uploaded_test_image.keys())[0]

Incarcati o imagine de test


Saving pexels-cottonbro-6770039.jpg to pexels-cottonbro-6770039.jpg


### Deschiderea și transformarea imaginii

In [ ]:
original_image = Image.open(uploaded_test_image_path).convert("RGB")
image_width, image_height = original_image.size
print(f"Width: {image_width} Height: {image_height}")
display(original_image)

transformed_image = transform(original_image).unsqueeze(0).to(DEVICE)



Output hidden; open in https://colab.research.google.com to view.

# Model convolutional simplu

## Grid 7 x 7 celule

In [ ]:
model_file = os.path.join(model_dir, 'grid_simple_model_7.pth')
if os.path.isfile(model_file):
  model_7 = GridSimpleCNNModel(griddim=7, image_size=224).to(DEVICE)
  model_7.load_state_dict(torch.load(model_file, map_location=torch.device('cpu'), weights_only=True))
  model_7.eval()
  with torch.no_grad():
      output = model_7(transformed_image)
      output = output.squeeze(0)  # Remove batch dim
      rows, cols = output.shape

  del model_7
  gc.collect()
  gridimage = getGridImage(original_image, output, grid_size=7)
  display(gridimage)


Output hidden; open in https://colab.research.google.com to view.

### Grid de 14 x 14 celule

In [ ]:
model_file = os.path.join(model_dir, 'grid_simple_model_14.pth')
if os.path.isfile(model_file):
  model_14 = GridSimpleCNNModel(griddim=14, image_size=224).to(DEVICE)
  model_14.load_state_dict(torch.load(model_file, map_location=torch.device('cpu'), weights_only=True))
  model_14.eval()
  with torch.no_grad():
      output = model_14(transformed_image)
      output = output.squeeze(0)  # Remove batch dim
      rows, cols = output.shape

  del model_14
  gc.collect()
  gridimage_14 = getGridImage(original_image, output, grid_size=14)
  display(gridimage_14)

Output hidden; open in https://colab.research.google.com to view.

### Grid de 21 x 21 celule

In [ ]:
model_file = os.path.join(model_dir, 'grid_simple_model_21.pth')
if os.path.isfile(model_file):
  model_21 = GridSimpleCNNModel(griddim=21, image_size=224).to(DEVICE)
  model_21.load_state_dict(torch.load(model_file, map_location=torch.device('cpu'), weights_only=True))
  model_21.eval()
  with torch.no_grad():
      output = model_21(transformed_image)
      output = output.squeeze(0)  # Remove batch dim
      rows, cols = output.shape

del model_21
gc.collect()
gridimage_21 = getGridImage(original_image, output, grid_size=21)
display(gridimage_21)

Output hidden; open in https://colab.research.google.com to view.

### Grid de 28 x 28 celule

In [ ]:
model_file = os.path.join(model_dir, 'grid_simple_model_28.pth')
if os.path.isfile(model_file):
  model_28 = GridSimpleCNNModel(griddim=28, image_size=224).to(DEVICE)
  model_28.load_state_dict(torch.load(model_file, map_location=torch.device('cpu'), weights_only=True))
  model_28.eval()
  with torch.no_grad():
      output = model_28(transformed_image)
      output = output.squeeze(0)  # Remove batch dim
      rows, cols = output.shape

del model_28
gc.collect()
gridimage_28 = getGridImage(original_image, output, grid_size=28)
display(gridimage_28)

Output hidden; open in https://colab.research.google.com to view.

## Model cu bloc convolutional de tip YOLO

### Grid de 7 x 7 celule

In [ ]:
model_file = os.path.join(yolo_model_dir, 'yolo_grid_model_224_7.pth')
if os.path.isfile(model_file):
  model_7 = GridYOLOCNNModel(griddim=7, image_size=224).to(DEVICE)
  model_7.load_state_dict(torch.load(model_file, map_location=torch.device('cpu'), weights_only=True))
  model_7.eval()
  with torch.no_grad():
      output = model_7(transformed_image)
      output = output.squeeze(0)  # Remove batch dim
      rows, cols = output.shape

  del model_7
  gc.collect()
  gridimage = getGridImage(original_image, output, grid_size=7)
  display(gridimage)

Output hidden; open in https://colab.research.google.com to view.

### Grid de 14 x 14 celule

In [ ]:
model_file = os.path.join(yolo_model_dir, 'yolo_grid_model_224_14.pth')
if os.path.isfile(model_file):
  model_14 = GridYOLOCNNModel(griddim=14, image_size=224).to(DEVICE)
  model_14.load_state_dict(torch.load(model_file, map_location=torch.device('cpu'), weights_only=True))
  model_14.eval()
  with torch.no_grad():
      output = model_14(transformed_image)
      output = output.squeeze(0)  # Remove batch dim
      rows, cols = output.shape

  del model_14
  gc.collect()
  gridimage = getGridImage(original_image, output, grid_size=14)
  display(gridimage)

Output hidden; open in https://colab.research.google.com to view.

### Grid de 21 x 21 celule

In [ ]:
model_file = os.path.join(yolo_model_dir, 'yolo_grid_model_224_21.pth')
if os.path.isfile(model_file):
  model_21 = GridYOLOCNNModel(griddim=21, image_size=224).to(DEVICE)
  model_21.load_state_dict(torch.load(model_file, map_location=torch.device('cpu'), weights_only=True))
  model_21.eval()
  with torch.no_grad():
      output = model_21(transformed_image)
      output = output.squeeze(0)  # Remove batch dim
      rows, cols = output.shape

  del model_21
  gc.collect()
  gridimage = getGridImage(original_image, output, grid_size=21)
  display(gridimage)

Output hidden; open in https://colab.research.google.com to view.

### Grid de 28 x 28 celule

In [ ]:
model_file = os.path.join(yolo_model_dir, 'yolo_grid_model_224_28.pth')
if os.path.isfile(model_file):
  model_28 = GridYOLOCNNModel(griddim=28, image_size=224).to(DEVICE)
  model_28.load_state_dict(torch.load(model_file, map_location=torch.device('cpu'), weights_only=True))
  model_28.eval()
  with torch.no_grad():
      output = model_28(transformed_image)
      output = output.squeeze(0)  # Remove batch dim
      rows, cols = output.shape

  del model_28
  gc.collect()
  gridimage = getGridImage(original_image, output, grid_size=28)
  display(gridimage)

Output hidden; open in https://colab.research.google.com to view.